<a href="https://colab.research.google.com/github/mamba124/just_for_fun/blob/master/Denoising_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Из кераса загружаем необходимые слои для нейросети
import tensorflow.keras.layers as layers
from tensorflow.keras import backend as K # подтягиваем базовые керасовские функции
import tensorflow.keras.optimizers as optimizers# загружаем выбранный оптимизатор
from tensorflow.keras import utils # загружаем утилиты кераса
from google.colab import files # модуль для загрузки файлов в colab
import matplotlib.pyplot as plt # из библиотеки для визуализации данных возьмём интерфейс для построения графиков простых функций
from tensorflow.keras.preprocessing import image # модуль для отрисовки изображения
import numpy as np # библиотека для работы с массивами данных
import librosa #Параметризация аудио
import os 

import pandas as pd # библиотека для анализа и обработки данных
from PIL import Image # модуль для отрисовки изображения
from sklearn.model_selection import train_test_split # модуль для разбивки выборки на тренировочную/тестовую
from sklearn.preprocessing import StandardScaler # модуль для стандартизации данных

from tensorflow.keras.models import Model, load_model #из кераса подгружаем абстрактный класс базовой модели, метод загрузки предобученной модели
from tensorflow.keras.optimizers import RMSprop #из кераса загружаем выбранный оптимизатор

#Автокодировщик

In [0]:
def baseAutoencoder(): # зададим функцию создания базового автокодировщика
    img_input = Input((28,28,1)) # задаём входные размеры

    x = Conv2D(32, (3, 3), padding='same', activation='relu')(img_input) # входные данные передаем на слой двумерной свёртки
    x = BatchNormalization()(x) # затем пропускаем через слой нормализации данных 
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x) # далее снова слой двумерной свёртки
    x = BatchNormalization()(x) # и еще слой нормализации данных
    x = MaxPooling2D()(x) # передаём на слой подвыборки, снижающий размерность поступивших на него данных

    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x) # передаем на слой двумерной свёртки
    x = BatchNormalization()(x) # пропускаем через слой нормализации данных 
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)  # далее снова слой двумерной свёртки
    x = BatchNormalization()(x) # и еще слой нормализации данных
    x = MaxPooling2D()(x) # передаём на слой подвыборки
    # Изображение ужали до 7*7

    x = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same', activation='relu')(x) # слой разжимает данные(с 7*7 на 14*14)
    x = BatchNormalization()(x) # слой нормализации данных
    
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x) # передаем на слой двумерной свёртки
    x = BatchNormalization()(x) # слой нормализации данных
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x) # ещё слой двумерной свёртки
    x = BatchNormalization()(x) # слой нормализации данных
    # Сжатие MaxPooling2D не применяем

    x = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same', activation='relu')(x) # слой разжимает данные(с 14*14 на 28*28)
    x = BatchNormalization()(x) # слой нормализации данных
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x) # передаем на слой двумерной свёртки
    x = BatchNormalization()(x) # слой нормализации данных
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x) # ещё слой двумерной свёртки
    x = BatchNormalization()(x) # слой нормализации данных

    # Финальный слой двумерной свертки, выдающий итоговое изображение
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) # указываем модель, с оригинальным изображением на входе в сеть и сжатым-разжатым на выходе из сети
    model.compile(optimizer=Adam(),
                  loss='mean_squared_error') # компилируем модель с оптимайзером Адам и среднеквадратичной ошибкой

    return model # функция вернёт заданную модель

# AUDIO

In [2]:
!pip install soundfile

In [0]:
import librosa # for mel-spectrogram estimation
import soundfile # for opening .flac audio
from matplotlib import pyplot as plt
import numpy as np

In [0]:
st= '/content/drive/My Drive/train/clean/47/47_122796_47-122796-0071 (1).npy'
stop = st[-7:]

In [0]:
def get_chunks(signal, length, step):
  batch =[]
  syms_len = len(signal)
  index = 0
  
  while (index + length <= syms_len):
    batch.append(signal[index:index+length])
    index += step

  return batch

def prepare_chunks(audio,length, step):
  batch_list = []
  #for item in audio:
   # batch_list.append(get_chunks(item, length, step))
  batch_list = get_chunks(audio, length, step)
  return np.array(batch_list)

In [0]:
directory = '/content/drive/My Drive/train/'
folders = os.listdir(directory)
clean_arrays = []
noisy_arrays = []
patch = os.listdir(directory+'clean/')

for folder in patch[:300]:
  for arr in os.listdir(directory+'clean/'+folder):
    if arr[-7:]!=stop:
      clean_arrays.append(np.load(directory+'clean/'+folder+'/'+arr))
      noisy_arrays.append(np.load(directory+'noisy/'+folder+'/'+arr))



In [6]:
clean_arrays_ = []
noisy_arrays_ = []

for i in range(len(clean_arrays[:-1])):
  for j in range(len(clean_arrays[i])):
    clean_arrays_.append(clean_arrays[i][j])
    noisy_arrays_.append(noisy_arrays[i][j])

NameError: ignored

In [0]:
np.save('c.npy', clean_arrays_)
np.save('n.npy', noisy_arrays_)

In [0]:
np.save('c1.npy', x_trainC)
np.save('n1.npy',x_trainN)

In [0]:
length = 60 
step = 30

x_trainC = prepare_chunks(clean_arrays_, length, step)
x_trainN = prepare_chunks(noisy_arrays_, length, step)



In [0]:
length = 60 
step = 30
x_trainC = np.load('c1.npy')
x_trainN = np.load('n1.npy')

In [0]:
y_trainC = np.ones((x_trainC.shape[0],1))
y_trainN = np.zeros((x_trainN.shape[0],1))

In [21]:
x_trainC[0]

array([[ 0.3887 ,  0.2979 ,  0.4097 , ..., -0.8145 , -0.833  , -0.848  ],
       [ 0.2964 ,  0.3025 ,  0.334  , ..., -0.8374 , -0.8276 , -0.8516 ],
       [ 0.119  ,  0.2399 ,  0.1682 , ..., -0.851  , -0.845  , -0.8433 ],
       ...,
       [ 0.6104 ,  0.4626 ,  0.3252 , ...,  0.03114,  0.0225 , -0.01971],
       [ 0.693  ,  0.7256 ,  0.854  , ..., -0.1829 , -0.2048 , -0.2715 ],
       [ 0.6665 ,  0.947  ,  1.076  , ..., -0.2837 , -0.287  , -0.337  ]],
      dtype=float16)

In [0]:
def shuffle(a, b):
    p = np.random.permutation(len(a)) 
    return a[p], b[p] 
x_train = np.concatenate([x_trainC, x_trainN], axis = 0)
y_train = np.concatenate([y_trainC, y_trainN], axis = 0)

x_train_, y_train_ = shuffle(x_train, y_train)
#x_val_, y_val_ = shuffle(x_val, y_val)

In [0]:
def classificator(vector):
  shape_ = (vector.shape[1],vector.shape[2])
  inputs = layers.Input(shape = shape_)
  hidden = layers.Conv1D(8, 3)(inputs)
  hidden = layers.Activation('tanh')(hidden)

  hidden = layers.Flatten()(hidden)
  hidden = layers.Dense(56)(hidden)
  hidden = layers.Activation('relu')(hidden)

  hidden = layers.Dense(100)(hidden)
  out = layers.Dense(1, activation = 'sigmoid')(hidden)
  
  model = Model(inputs = inputs, outputs = out)

  return model

def autoencoder(vector):

  inputs_vector = layers.Input(shape = (vector.shape[1],vector.shape[2]))

  out_class = layers.BatchNormalization()(inputs_vector)
  out_class= layers.Conv1D(64, 3, padding = 'same')(out_class)
  out_class = layers.Flatten()(out_class)
  out_class = layers.Dense(100, activation='elu')(out_class)
  out_class = layers.Dense(56, activation = 'relu')(out_class)
  out_class = layers.Dense(1, activation = 'sigmoid')(out_class)
  
  ##Block1##
  hidden = layers.Conv1D(2, 3, padding = 'same') (inputs_vector)
  hidden = layers.Activation('tanh')(hidden)
  hidden = layers.BatchNormalization()(hidden)

  hidden = layers.Conv1D(2, 3, padding = 'same')(hidden)
  hidden = layers.Activation('relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)

  hidden = layers.MaxPooling1D(3)(hidden)

  
  ##Block2##
  hidden = layers.Conv1D(4, 3, padding = 'same') (hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.Conv1D(4, 3, padding = 'same', strides = 2)(hidden)
  hidden = layers.Activation('relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.MaxPooling1D(3)(hidden)
  
  ##Block3##
  hidden = layers.Conv1D(128, 3, padding = 'same') (hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.MaxPooling1D(3)(hidden)  
  
  ##UPBlock1##
  hidden = layers.UpSampling1D(3)(hidden)
  hidden = layers.Conv1D(64, 3, padding = 'same', activation= 'relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.Multiply()([out_class, hidden])

  ##UPBlock2##
  hidden = layers.UpSampling1D(4)(hidden)
  hidden = layers.Conv1D(64, 3, padding = 'same', activation= 'relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.Multiply()([out_class, hidden])

  ##UPblock3##
  hidden = layers.UpSampling1D(5)(hidden)
  hidden = layers.Conv1D(64, 3, padding = 'same',  activation= 'relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.Multiply()([out_class, hidden])

  out = layers.Conv1D(80, 3, padding = 'same',  activation= 'tanh')(hidden)

  modelD = Model(inputs=inputs_vector, outputs = [out, out_class])
  #modelD.summary()

  return modelD


  

In [0]:
denoise = autoencoder(x_train_)
denoise.compile(optimizer = RMSprop(), loss = ['mae','binary_crossentropy'], metrics = ['mse','accuracy'])
denoise.fit(x_train_, [x_train_, y_train_], validation_split = 0.1, batch_size=500, epochs = 50)

Epoch 1/50
416/416 [==============================] - 41s 99ms/step - loss: 0.5687 - conv1d_155_loss: 0.2624 - dense_65_loss: 0.3063 - conv1d_155_mse: 0.1150 - conv1d_155_accuracy: 0.0955 - dense_65_mse: 0.0890 - dense_65_accuracy: 0.8802 - val_loss: 0.4578 - val_conv1d_155_loss: 0.2421 - val_dense_65_loss: 0.2157 - val_conv1d_155_mse: 0.0947 - val_conv1d_155_accuracy: 0.0965 - val_dense_65_mse: 0.0642 - val_dense_65_accuracy: 0.9121
Epoch 2/50
416/416 [==============================] - 40s 97ms/step - loss: 0.4135 - conv1d_155_loss: 0.2143 - dense_65_loss: 0.1992 - conv1d_155_mse: 0.0776 - conv1d_155_accuracy: 0.1125 - dense_65_mse: 0.0572 - dense_65_accuracy: 0.9236 - val_loss: 0.3949 - val_conv1d_155_loss: 0.2123 - val_dense_65_loss: 0.1826 - val_conv1d_155_mse: 0.0765 - val_conv1d_155_accuracy: 0.1225 - val_dense_65_mse: 0.0528 - val_dense_65_accuracy: 0.9285
Epoch 3/50
416/416 [==============================] - 40s 95ms/step - loss: 0.3724 - conv1d_155_loss: 0.2071 - dense_65_loss

In [50]:
#classificator = classificator(x_train_)
classificator.compile(optimizer = RMSprop(), loss = 'binary_crossentropy', metrics = 'accuracy')
classificator.fit(x_train_, y_train_, batch_size=500, epochs=20, validation_splt = 0.1)

Epoch 1/20
463/463 [==============================] - 6s 13ms/step - loss: 0.3255 - accuracy: 0.8609
Epoch 2/20
425/463 [==========================>...] - ETA: 0s - loss: 0.2520 - accuracy: 0.8977

KeyboardInterrupt: ignored

In [0]:
denoised, classes = denoise.predict(x_train_[-300:])

In [0]:
def glue_chunks(signal_list):
  signals = []
  for signal in signal_list:
    glued = []
    for i in range(len(signal)):
      if i == 0: ind = 0
      else: ind = length - step
      for j in range(ind,length):
        glued.append(signal[i][j])
    signals.append(np.array(glued))

  signals = np.array(signals)

denoised_glued = glue_chunks(denoised)

In [0]:
for i, record in enumerate(denoised_glued): 
  np.save('/final/' + i, record)